<h1>Practice</h1>

<a href="https://scikit-learn.org/0.19/tutorial/text_analytics/working_with_text_data.html">Link</a>

<h2>Scikit-learn tutorial on Working With Text Data</h2>


The goal of this guide is to explore some of the main scikit-learn tools on a single practical task: analysing a collection of text documents (newsgroups posts) on twenty different topics.

In this section we will see how to:

* load the file contents and the categories
* extract feature vectors suitable for machine learning
* train a linear model to perform categorization
* use a grid search strategy to find a good configuration of both the feature extraction components and the classifier

In [1]:
from sklearn.datasets import fetch_20newsgroups

In [2]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

In [3]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

The returned dataset is a <mark>scikit-learn</mark> “bunch”: a simple holder object with fields that can be both accessed as python <mark>dict</mark> keys or <mark>object</mark> attributes for convenience, for instance the <mark>target_names</mark> holds the list of the requested category names:

In [4]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

The files themselves are loaded in memory in the `data` attribute. For refernce the filenames are also available:

In [5]:
len(twenty_train.data), len(twenty_train.filenames)

(2257, 2257)

In [6]:
#Print the first lines of the first loaded file:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [7]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


Supervised learning algorithms will require a category label for each document in the training set.
In this case the category is the name of the newsgroup which 
also happens to be the name of the folder holding the individual documents.
<br>
For speed and space efficiency reasons `scikit-learn` loads the target attribute as an array of integers that corresonds to the index of the category name in the <mark>traget_names</mark>list. The category integer id of each sample is stored in the `target` attribute:

In [8]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

It is possible to get back the category names as follows:

In [9]:
for t in twenty_train.target[:10]:
    print (twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


<h2>Extracting features from text files</h2><br>

In order to perform machine learining on text documents, we first need to turn the text into numerical feature vectors.<br>

<mark><h3>Bags of words</h3></mark>

The most intuitive way to do so is the bags of words representation:

<ol type="1">
    <li>assign a fixed id to each word occurring in any document  of the training set (for instance by building a dictionary from words to integer indices)</li>
    <li>for each document <mark>#i</mark>, count the number of occurrences of each word 'w' and store it in <mark>X[i, j]</mark> as the value of feature '#j' where <mark>j</mark> is the index of word 'w' in the dictionary.</li>
</ol>

The bags of words representation implies that 'n_features' is the number of distinct words in the corpus: this is typically larger than 100,000.

If 'n_samples == 10000, storing 'x' as a numpy array of type float32 would require 10000 x 100000 x 4 bytes = 4GB in RAM which is barely manageable on today's computers.
<br>

Fortunately, <strong>most values in X will be zeros</strong> since for a given document less than a couple thousands of distinct words will be used. For this reason we say that bags of words are typically <strong>high-dimensional sparse datasets</strong>. We can save a lot of memory by only storing the non-zero parts of the feature vectors in memory.
<br>
<mark>scipy.sparse</mark> matrices are data structures that do exactly this, and 'scikit-learn' has built-in support for these structures.

    

<h2> Tokenizing text with  scikit-learn </h2>
<br>
Text preporcessing, tokenizing and filtering of stopwords are included in a high level component that is able to build a dictionary of features and transform documents to features vectors.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(twenty_train.data)
x_train_counts.shape

(2257, 35788)

<strong>CountVectorizer</strong> supports counts of N-grams of words or consecutive characters. Once fitted, the vectorizer has built a dictionary of feature indices:

In [11]:
count_vect.vocabulary_.get(u'algorithm')

4690

The index value of a word in the vocabulary is linked to its frequency in the whole training corpus.

<h2>From occurrences to frequencies</h2>
Occurrence count is a good start but there is an issue: longer documents will have higher average count values than shorter doucments, even though they might talk about the same topics.

More descripition on `tf` "Terms Frequencies" and `tf-idf` "Term Frequency times inverse Document Frequency" back in  <a href="https://scikit-learn.org/0.19/tutorial/text_analytics/working_with_text_data.html">Link</a> <br>
Both `tf` and `tf-idf` can be computed as follows:

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(x_train_counts)
x_train_tf = tf_transformer.transform(x_train_counts)
x_train_tf.shape

(2257, 35788)

In [13]:
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)
x_train_tfidf.shape

(2257, 35788)

<h2>Training a  classifier</h2>

Now that we have our features, we can train a classifier to try to predict the category of a post. Let's start with a <a href="https://scikit-learn.org/0.19/modules/naive_bayes.html#naive-bayes">Naive Byase Classifier</a>, which provides a nice baseline fo this task. <mark>scikit-learn</mark> includes several variants of this calssifier; the one most suitable for word coutns is the multinomial variant:

In [14]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(x_train_tfidf, twenty_train.target)

In [15]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
x_new_counts = count_vect.transform(docs_new)
x_new_tfidf = tfidf_transformer.transform(x_new_counts)

predicted = clf.predict(x_new_tfidf)

for doc, category in zip( docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


<h2>Building a pipeline</h2>
In order to make the vectorizer => transformer => classifier easier to work with, <mark>scikit-learn</mark> provides a 'Pipeline' class that behaves like a compound classifier

In [16]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultinomialNB()),
                    ])
# The names 'vect', 'tfidf' and clf (classifier) are arbitrary. We shall see their use in the section on grid search, below.
# We can now train the model with a single command:

In [17]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

<h2> Evaulation of the performance on the test set</h2>

In [18]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8348868175765646

We achived 83.4% accuracy. Let's see if we can do better with a linear <a href="https://scikit-learn.org/0.19/modules/svm.html#svm">support vector machine(SVM)</a>, which is widely regarded as one of the best text classification algorithms (although it's also ba bit slower than naive Bayes). We can change the learner by just plugging a different classifier object into our pipeline:

In [19]:
# Pipeline with Niave Bayes classier (OLD) copied from the above:
# from sklearn.pipeline import Pipeline
# from sklearn.naive_bayes import MultinomialNB
# text_clf = Pipeline([('vect', CountVectorizer()),
#                     ('tfidf', TfidfTransformer()),
#                     ('clf', MultinomialNB()),
#                     ])

# Pipeline with Support Vector Machine Classifier:
# from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                          alpha=1e-3, random_state=42,
                                          max_iter=5, tol=None)),
                    ])

In [20]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [21]:
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.9101198402130493

`scikit-learn` further provides utilities for more detailed performance analysis of the results:

In [22]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted, target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.80      0.87       319
         comp.graphics       0.87      0.98      0.92       389
               sci.med       0.94      0.89      0.91       396
soc.religion.christian       0.90      0.95      0.93       398

              accuracy                           0.91      1502
             macro avg       0.91      0.91      0.91      1502
          weighted avg       0.91      0.91      0.91      1502



In [23]:
metrics.confusion_matrix(twenty_test.target,predicted)

array([[256,  11,  16,  36],
       [  4, 380,   3,   2],
       [  5,  35, 353,   3],
       [  5,  11,   4, 378]], dtype=int64)

As expected, the confusion matrix shows that posts from the newsgroups on atheism and christian are more often confused for one another than with computer graphics.

<h2>Parameter tuning using grid search</h2>

We've already encountered some parameters such as `use_idf` in the `TfidfTransformer`. Classifiers tend to have many parameters as well; e.g. `MultinomialNB` includes a smoothing parameter `alpha` and `SGDClassifier` has a penalty parameter `alpha` and configurable loss and penalty terms in the objective function (see the module documentation, to get a description of these).

Instead of tweaking the parameters of the various components of the chain, it is possible to run an exhaustive search of the best parameters on a grid of possible values. We try out all classifiers on either words or bigrams, with or without idf, and with a penalty parameter of either 0.01 or 0.001 fo the linear SVM:

In [24]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1,1), (1,2)],
             'tfidf__use_idf': (True, False),
             'clf__alpha': (1e-2, 1e-3)}

Obviously such an exhaustive search can be expensive. If we have multiple CPU cores at our disposal, we can tell the grid searcher to try these eight parameter combinations in parallel with the `n_jobs` parameter. If we give parameter a value of `-1` grid search will detect how many cores are installed and uses them all:

In [25]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

The grid search instance behaves like a normal `scikit-learn` model. Let's perform the search on a smaller subset of the training data to speed up the computation:

In [26]:
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

The result of calling `fit` on a `GridSearchCV` object is a classifier that we cna use to `predict`:

In [27]:
twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

'soc.religion.christian'

The objects `best_score_` and `best_params_` attributes sotre the bes mean score and the parameters setting corresponding to that score:

In [28]:
gs_clf.best_score_

0.9175000000000001

In [29]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)


A more detailed summary of the search is available at `gs_clf.cv_results_`. <br>
The `cv_results_` parameter can be easily imported into pandas as a `DataFrame` for further inspection.

<h2>Exercises</h2>
All exercises can be found <a href="https://scikit-learn.org/0.19/tutorial/text_analytics/working_with_text_data.html">here</a>

<h3>Exercise 1: Language identification</h3>
<ol type="1">
<li>Write a text classification pipeline using a custom preprocessor and CharNGramAnalyzer using data from Wikipedia articles as training set.</li>

<li>Evaluate the performance on some held out test set.</li>
</ol>

<h3>Exercise 2: Sentiment Analysis on movie reviews</h3>
<ol type="1">
<li>Write a text classification pipeline to classify movie reviews as either positive or negative.</li>
<li>Find a good set of parameters using grid search.</li>
<li>Evaluate the performance on a held out test set.</li>
</ol>

<h3>Exercise 3: CLI text classification utility</h3>
sing the results of the previous exercises and the cPickle module of the standard library, write a command line utility that detects the language of some text provided on stdin and estimate the polarity (positive or negative) if the text is written in English.

Bonus point if the utility is able to give a confidence level for its predictions.

<h3>Where to from here</h3>
Here are a few suggestions to help further your scikit-learn intuition upon the completion of this tutorial:
<ul>
<li>Try playing around with the analyzer and token normalisation under CountVectorizer</li>
<li>If you don’t have labels, try using Clustering on your problem.</li>
<li>If you have multiple labels per document, e.g categories, have a look at the Multiclass and multilabel section</li>
<li>Try using Truncated SVD for latent semantic analysis.</li>
<li>Have a look at using Out-of-core Classification to learn from data that would not fit into the computer main memory.</li>
<li>Have a look at the Hashing Vectorizer as a memory efficient alternative to CountVectorizer.</li>
</ul>